# Warnings

In [ ]:
import warnings
import os
import torch
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from transformers import logging
logging.set_verbosity_error()
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf


In [2]:
import sys
sys.version

'3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]'

# DataClass

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np


class AnyData(Dataset):
    
    def __init__(self):
        # xy = np.loadtxt('', delimiter='', dtype=np.float32, skiprows=1)
        # self.x
        # self.y
        # self.n_namples = xy.shape[0]
        pass
    
    def __getitem__(self, index):
        # return self.x[index], self.y[index]
        pass
        
    def __len__(self):
        # return self.n_namples
        pass
    
dataset = AnyData()
# dataset[0]
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True, num_workers=2)
# dataiter = iter(dataloader)
# data_1 = dataiter.next()

In [9]:
import torch
x = torch.eye(10)
x.sum().item()

10.0

# Vectors

### Bow

In [ ]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
# counts = Counter(words)
# cv = CountVectorizer(vocabulary=counts.keys())
# cv.fit_transform(i).toarray()[0].tolist()


vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

### tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=0.0005,
    ngram_range=(1, 2)
)
x = vectorizer.fit_transform(data)

### split & shuffle

In [ ]:
def split_train_test(corpus, labels, ratios=False):
    X_train, X_test, y_train, y_test = [], [], [], []
    if ratios:
        for k, v in ratios.items():
            train_rate = v[1] / v[0]
            corpus_current, labels_current = data_by_class_selection(corpus, labels, k)
            x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(
                corpus_current, 
                labels_current, 
                test_size=train_rate, 
                random_state=41)
            X_train.extend(x_train_1)
            X_test.extend(x_test_1)
            y_train.extend(y_train_1)
            y_test.extend(y_test_1)
    else:
        for i in dict(pd.Series(labels).value_counts()).keys():
            corpus_current, labels_current = data_by_class_selection(corpus, labels, i)
            train_rate = 0.3
            x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(
                corpus_current, 
                labels_current, 
                test_size=train_rate, 
                random_state=41)
            X_train.extend(x_train_1)
            X_test.extend(x_test_1)
            y_train.extend(y_train_1)
            y_test.extend(y_test_1)
    return X_train, X_test, y_train, y_test

def data_by_class_selection(corpus, labels, c):
    new_corpus = []
    new_labels = []
    for d, l in zip(corpus, labels):
        if l == c:
            new_corpus.append(d)
            new_labels.append(l)
    return new_corpus, new_labels

def shuffle_data(corpus, labels):
    combined = list(zip(corpus, labels))
    random.shuffle(combined)
    corpus[:], labels[:] = zip(*combined)
    return corpus, labels

### preprocess

In [71]:
from pymystem3 import Mystem
import re
stemmer = Mystem()
def preprocess(t, stem=True):
    x = ''.join([i for i in t.lower() if i in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя .'])  # <<<<<<< LOWER
    x = x.replace('.', ' ')
    if stem:
        x = ''.join(stemmer.lemmatize(x))
    else:
        pass
    x = ' '.join([i for i in x.split() if len(i) > 2])
    x = re.sub('\s+', ' ', x)
    return x.strip()

In [70]:
def clean_tesseract(x):
    import re
    x = re.sub(r'\"', ' ', x)
    x = x.replace('\\n', ' ')
    x = x.replace('\\', ' ')
    x = re.sub('\s+', ' ', x)
    return x.strip()

# Pipelines (classic ML)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
folds = 2

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer = 'word', min_df=0)),#, max_df=0.9, max_features=10000)),
    # ('LinearSVC', LinearSVC(C=1, random_state=42))
    ('LogisticRegression', LogisticRegression(solver='liblinear', random_state=42))
    # ('SGDClassifier', SGDClassifier(loss='hinge', alpha = 0.001))# np.linspace(0.01, 0.05, 10)
])
parameters = { 
    # 'tfidf__max_df': (0.9, 1), # 0.8 <- 0.95 ### 0.6 
    # 'tfidf__min_df': (0), # 0.01 <- 0.05 ### 0.005 <- 0.01 ### 0.001
    'tfidf__ngram_range': [(1, 2), (1, 3)]
    # 'svm__decision_function_shape': ['ovo', 'ovr'],
    # 'svm__C': (4, 6)
    # 'tfidf__max_features': (10000, 32000)
}
c_val = StratifiedKFold(n_splits=folds)

grid_search_tune = GridSearchCV(pipeline, parameters, cv=c_val, verbose=3, n_jobs=4, scoring='f1_macro')
grid_search_tune.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

In [ ]:
from sklearn.base import BaseEstimator
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from imblearn.under_sampling import RandomUnderSampler


class TextClassifier(BaseEstimator):
    @staticmethod
    def _build_model():
        model = Pipeline([
            ('tfidf', TfidfVectorizer(analyzer='word', min_df=0, max_df=0.6, ngram_range=(1, 3))),
            ('RandomUnderSampler', RandomUnderSampler()),
            ('Smote', SMOTE()),
            ('svm', SVC(C=3, kernel='linear', decision_function_shape='ovr'))
        ])

        return model

    def __init__(self):
        self.model = TextClassifier._build_model()

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

    def predict_proba(self, X):
        return self.model.predict_proba(X)

    def save(self, path):
        self.model.save(path)

    def load(self, path):
        return self.model.load_model(path)
model = TextClassifier()._build_model()

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer = 'word', min_df=0, max_df=0.6, ngram_range=(1, 3))),
    ('RandomUnderSampler', RandomUnderSampler()),
    ('Smote', SMOTE()),
    ('svm', SVC(C=3, kernel='linear', decision_function_shape='ovr'))
])

# Quality

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
m = confusion_matrix(y_true, y_pred)
new_m = []
l = []
print(m)
for i in m:
    s = sum(i)
    for ii in i:
        l.append(round(ii / s * 100))
    new_m.append(l)
    l = []
print(np.array(new_m))

print(f1_score(y_true, y_pred, average='macro'))
print(accuracy_score(y_true, y_pred))

In [ ]:
print(__doc__)
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    
cnf_matrix = np.array(new_m).astype(int)

    
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(num=None, figsize=(13, 11), dpi=80, facecolor='w', edgecolor='k')
plot_confusion_matrix(cnf_matrix, classes=list(pd.Series(y_pred).value_counts().keys()),
                      title='Confusion matrix huyatrix')

plt.show()

### Jaccard index

In [1]:
def jaccard(x1, x2):
    intersection = set(x1.split()).intersection(set(x2.split()))
    union = set(x1.split()).union(set(x2.split()))
    try:
        return float(format(len(intersection) / len(union), '.2f'))
    except ZeroDivisionError:
        return 233.
jaccard('Дяденька король цветной капусты хотца', 'Дяденька король цветной капусты')

0.8

# Open

In [9]:
def get_files(path, ext):
    import os
    paths_to_files = []
    for root, dir, files in os.walk(path):
        if files:
            for f in files:
                if ext in f:
                    paths_to_files.append(os.path.join(root, f))
    if paths_to_files:
        return paths_to_files
    else:
        raise ValueError('No such directory')
# get_files('<path>', 'csv')

In [ ]:
try:
    with open('personal_info_wall_media_sep152020.json') as json_file:
        data = json.load(json_file)
    data.update(user_data)
    json.dump(data, open("personal_info_wall_media_sep152020.json", 'w'))
except FileNotFoundError:
    json.dump(user_data, open("personal_info_wall_media_sep152020.json", 'w'))
user_data = {}

In [ ]:
import csv
new_labels1, new_corpus1 = [], []
with open('type_march_.csv', 'rt') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for n, i in enumerate(readCSV):
        # print(i)
        # break
        if n != 0:
            new_labels1.append(i[0])
            new_corpus1.append(i[1])

# Notes

In [ ]:
flat_list = [item for sublist in t for item in sublist]
{k: v for k, v in sorted(result.items(), key=lambda item: item[1])}

In [ ]:
from copy import copy
def top_n_max(series, top=2, diraction=max):
    series = list(set(series.tolist()))
    if diraction == max:
      top_result = sorted(series)[-top:]
    else:
      top_result = sorted(series)[:top]
    return top_result